Initial installs, updating working directory

In [ ]:
# Installing the transformers library
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 16.6 MB/s eta 0:00:00


In [ ]:
!pip install -q -U "tensorflow-text==2.8.*"
!pip install scikit-multilearn
!pip install sklearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.0/498.0 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 KB 22.5 MB/s eta 0:00:00


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
from google.colab import files
uploaded = files.upload()

Saving go_emotions_dataset.csv to go_emotions_dataset.csv


# New section

In [ ]:
# Updating the working directory
!pwd
!cd /content/drive/MyDrive/'Colab Notebooks'/OneDriveCopy/Practice/ML/NLP/EDA
!pwd

/content
/content


# Imports

In [ ]:
import os
import sys
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from skmultilearn.model_selection import IterativeStratification, iterative_train_test_split
from sklearn import metrics
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

import transformers
from transformers import AutoTokenizer, BloomTokenizerFast, BloomForSequenceClassification, BloomForTokenClassification, AutoModelForSequenceClassification

Data prep

In [ ]:
# Read the data file
df = pd.read_csv(io.StringIO(uploaded['go_emotions_dataset.csv'].decode('utf-8')))
#df = pd.read_csv('go_emotions_dataset.csv')
df.head(10)

,id,text,example_very_unclear,admiration,amusement,anger,annoyance,approval,caring,confusion,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,eew5j0j,That game hurt.,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,eemcysk,>sexuality shouldn’t be a grouping category I...,True,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ed2mah1,"You do right, if you don't care then fuck 'em!",False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,eeibobj,Man I love reddit.,False,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,eda6yn6,"[NAME] was nowhere near them, he was by the Fa...",False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,eespn2i,Right? Considering it’s such an important docu...,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,eczuekb,"He isn't as big, but he's still quite popular....",False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,ed5tx8y,That's crazy; I went to a super [RELIGION] hig...,False,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,ef961hv,that's adorable asf,False,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,edl7cr3,"""Sponge Blurb Pubs Quaw Haha GURR ha AAa!"" fin...",False,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
print(df['example_very_unclear'].iloc[0]==0)
print(df.shape)
print(df.head(1))

True
(211225, 31)
        id             text  example_very_unclear  admiration  amusement  \
0  eew5j0j  That game hurt.                 False           0          0   

   anger  annoyance  approval  caring  confusion  ...  love  nervousness  \
0      0          0         0       0          0  ...     0            0   

   optimism  pride  realization  relief  remorse  sadness  surprise  neutral  
0         0      0            0       0        0        1         0        0  

[1 rows x 31 columns]


In [ ]:
# Dropping the 'example very unclear' column
df = df.loc[df['example_very_unclear']==0]
df = df.drop("example_very_unclear",axis =1)
print(df.shape)
print(df.head(1))

(207814, 30)
        id             text  admiration  amusement  anger  annoyance  \
0  eew5j0j  That game hurt.           0          0      0          0   

   approval  caring  confusion  curiosity  ...  love  nervousness  optimism  \
0         0       0          0          0  ...     0            0         0   

   pride  realization  relief  remorse  sadness  surprise  neutral  
0      0            0       0        0        1         0        0  

[1 rows x 30 columns]


In [ ]:
# Removing duplicate text
df.drop_duplicates(subset=["text"], inplace = True)
print(df.shape)

(57730, 30)


In [ ]:
# Assigning emotions to classes
classes = df.columns.values[2:]
classes

array(['admiration', 'amusement', 'anger', 'annoyance', 'approval',
       'caring', 'confusion', 'curiosity', 'desire', 'disappointment',
       'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
       'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism',
       'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise',
       'neutral'], dtype=object)

In [ ]:
# Capturing labels for rows of text
num_cols = df._get_numeric_data()
lab_df = num_cols.idxmax(axis=1)
df = pd.concat([df,lab_df],axis=1)
df.rename(columns={0:'emotion'},inplace=True)
df = df[['id', 'text','emotion']]
df.head(5)

,id,text,emotion
0,eew5j0j,That game hurt.,sadness
2,ed2mah1,"You do right, if you don't care then fuck 'em!",neutral
3,eeibobj,Man I love reddit.,love
4,eda6yn6,"[NAME] was nowhere near them, he was by the Fa...",neutral
5,eespn2i,Right? Considering it’s such an important docu...,gratitude


In [ ]:
# Counting samples of each class
df.groupby('emotion').size()

emotion
admiration         4874
amusement          2555
anger              2208
annoyance          3057
approval           4303
caring             1358
confusion          1812
curiosity          2230
desire              803
disappointment     1723
disapproval        2445
disgust             866
embarrassment       437
excitement         1201
fear                694
gratitude          2530
grief               133
joy                1383
love               1543
nervousness         241
neutral           15803
optimism           1371
pride               187
realization        1343
relief              239
remorse             432
sadness            1013
surprise            946
dtype: int64

Defining x, y

In [ ]:
#####################################################################
# Shuffling the data before splitting
df = df.sample(frac = 1, random_state=101).reset_index(drop=True)

In [ ]:
# Defining x and y
x = df['text']
y = df['emotion']

In [ ]:
##########################################################################
# Iterative stratified sampling
#x_train, y_train, x_test, y_test = iterative_train_test_split(x, y, test_size = 0.2)
x_train, y_train, x_test, y_test = train_test_split(x, y, test_size = 0.2)

In [ ]:
x_train[0]

'[NAME] do not have a profound love of mayo, where did you get that idea?'

In [ ]:
x_train.values[0]

'Ah, yes, the good old “glass elevator”'

In [ ]:
x_train

13945               Ah, yes, the good old “glass elevator”
27713    He will be perfectly fine besides being very n...
21030    My favourite part is that the Saints are compl...
22219    I have heard nothing about him this window. If...
14759    Nice rifle. I have a T3 in 7mm rem mag. N Mine...
                               ...                        
39827                           Gorgeous cats, poor babies
46342    Why not touch them? then don't shove people ou...
11027    I responded to someone else in this thread as ...
41979    Maybe focus on today instead of worrying about...
1510     The "extinction events" is lame because Earth ...
Name: text, Length: 46184, dtype: object

Tokenizer

In [ ]:
##############################################################################################
MODEL_NAME = 'bigscience/bloom-560m'
MAX_LENGTH = 256
PROBLEM = "multi_label_classification"
TENSOR = "pt"
NUM_LABEL = len(y)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, return_tensor=TENSOR)

# Tokenizing the text
#train_encodings = tokenizer(list(x_train.values),
#                            max_length=MAX_LENGTH,
#                            truncation=True, 
#                            padding=True)

#test_encodings = tokenizer(list(x_test.values),
#                            max_length=MAX_LENGTH,
#                            truncation=True, 
#                            padding=True)

In [ ]:
x_train[0]

'You are awesome, i envy you'

In [ ]:
x_train.values[0]

'"PETA approved" I\'m out. I hate PETA even more than I hate mlms'

In [ ]:
train_encodings[0]

Encoding(num_tokens=256, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
MAX_LEGTH = 256

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in x:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,
                        max_length=MAX_LENGTH,
                        truncation=True,
                        padding=True
                   )
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

# Print sentence 0, now as a list of IDs.
print('Original: ', x[0])
print('Token IDs:', input_ids[0])

Original:  You are awesome, i envy you
Token IDs: [5448, 1306, 158533, 15, 707, 12672, 92, 1152]


In [ ]:
print('Max sentence length: ', max([len(sen) for sen in input_ids]))

Max sentence length:  256


In [ ]:
MAX_LEN = 256

#Padding the input to the max length that is 64
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

In [ ]:
# Creating the attention masks
attention_masks = []

# For each sentence...
for sent in input_ids:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

Train-test split

In [ ]:
len(attention_masks)

57730

In [ ]:
attention_masks[0]

[1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
input_ids[0]

[5448, 1306, 158533, 15, 707, 12672, 92, 1152]

In [ ]:

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, classes, 
                                                            random_state=101, test_size=0.2)
# Performing same steps on the attention masks
train_masks, validation_masks, _, _ = train_test_split(attention_masks, classes,
                                             random_state=101, test_size=0.2)

Model Training

In [ ]:
# Defining the model
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, problem_type=PROBLEM, num_labels=NUM_LABEL)

Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at bigscience/bloom-560m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.train()

BloomForSequenceClassification(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0): BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
      (1): BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1

In [ ]:
########################################################
model.train()
#model.cuda()

with torch.no_grad():
    logits = model(**inputs).logits

Predictions

In [ ]:
##Turning into tensorflow datasets
emotion_count = len(y)

train_dataset = tf.data.Dataset.from_tensor_slices((dict(input_ids),y.values.reshape(-1,emotion_count)))
#test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings),y_test.values.reshape(-1,emotion_count)))

In [ ]:
input_ids = []

for sent in list(x_train.values):
  input_ids.append(tokenizer.encode(sent))


 # Creating the attention masks
attention_masks = []

# For each sentence...
for sent in input_ids:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask) 

NameError: ignored

In [ ]:
MAX_LEN = 256

#Padding the input to the max length that is 64
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

In [ ]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=101, test_size=0.1)
# Performing same steps on the attention masks
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=101, test_size=0.1)

ValueError: ignored

In [ ]:
#Converting the input data to the tensor , which can be feeded to the model
train_inputs = torch.tensor(input_ids)
train_inputs = train_inputs.unsqueeze(0)
#attention_mask = attention_mask.unsqueeze(0)
#validation_inputs = torch.tensor(validation_inputs)

#train_labels = torch.tensor(train_labels)
#validation_labels = torch.tensor(validation_labels)

#train_masks = torch.tensor(train_masks)
#validation_masks = torch.tensor(validation_masks)

NameError: ignored

In [ ]:
outputs = model(train_dataset)

AttributeError: ignored

In [ ]:


with torch.no_grad():
    logits = model(train_encodings).logits

predicted_class_ids = torch.arange(0, logits.shape[-1])[torch.sigmoid(logits).squeeze(dim=0) > 0.5]

AttributeError: ignored

In [ ]:
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_ids = torch.arange(0, logits.shape[-1])[torch.sigmoid(logits).squeeze(dim=0) > 0.5]

# To train a model on `num_labels` classes, you can pass `num_labels=num_labels` to `.from_pretrained(...)`
num_labels = len(model.config.id2label)
model = BloomForSequenceClassification.from_pretrained(
    "bigscience/bloom-560m", num_labels=num_labels, problem_type="multi_label_classification"
)

In [ ]:
from transformers import AutoTokenizer, BloomForTokenClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
model = BloomForTokenClassification.from_pretrained("bigscience/bloom-560m")

inputs = tokenizer(
    "HuggingFace is a company based in Paris and New York", add_special_tokens=False, return_tensors="pt"
)

with torch.no_grad():
    logits = model(**inputs).logits

predicted_token_class_ids = logits.argmax(-1)

# Note that tokens are classified rather then input words which means that
# there might be more predicted token classes than words.
# Multiple token classes might account for the same word
predicted_tokens_classes = [model.config.id2label[t.item()] for t in predicted_token_class_ids[0]]

labels = predicted_token_class_ids
loss = model(**inputs, labels=labels).loss

In [ ]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_ids = torch.arange(0, logits.shape[-1])[torch.sigmoid(logits).squeeze(dim=0) > 0.5]

# To train a model on `num_labels` classes, you can pass `num_labels=num_labels` to `.from_pretrained(...)`
num_labels = len(model.config.id2label)
model = BloomForSequenceClassification.from_pretrained(
    "bigscience/bloom-560m", num_labels=num_labels, problem_type="multi_label_classification"
)

labels = torch.sum(
    torch.nn.functional.one_hot(predicted_class_ids[None, :].clone(), num_classes=num_labels), dim=1
).to(torch.float)
loss = model(**inputs, labels=labels).loss

Metrics

In [ ]:
y_pred = xg_reg.predict(test_x)
y_prob = xg_reg.predict_proba(test_x)
# Calculate RMSE and accuracy
accuracy = accuracy_score(test_y, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
precision = metrics.precision_score(y_true, y_pred)
recall = metrics.recall_score(y_true, y_pred)
f1 = metrics.f1_score(y_true, y_pred)

NameError: ignored